In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import json
import altair as alt
from altair_saver import save

In [2]:
WD = "/Users/user/Desktop/v.03-Results"

#PATH = os.path.join(WD, "output")

PATH = WD

JSON_FILES = [os.path.join(PATH, file) for file in os.listdir(PATH) if file.endswith("_FEL.json") and os.path.isfile(os.path.join(PATH, file))]

pvalueThreshold = 0.1

print("# Number of FEL json files", len(JSON_FILES))

# Number of FEL json files 25


In [39]:
def get_JSONData(json_file):
    if os.stat(json_file).st_size == 0:
         print('# File is empty:', json_file)
    else:
        #print('File is not empty')
        with open(json_file, "r") as in_d:
             json_data = json.load(in_d)
        #end with
        in_d.close()
        return json_data
    #end if
#end method


def num_positive_sites(json_data, pvalueThreshold):
    data = json_data["MLE"]["content"]["0"]
    columns = json_data["MLE"]["headers"]
    headers = [x[0] for x in columns]
    df = pd.DataFrame(data, columns=headers, dtype = float)
    df["omega"] = df["beta"] / df["alpha"]
    df_results = df[df["p-value"] <= pvalueThreshold]
    
    # 
    positive_sites = df_results[df_results["omega"] > 1.0]
    return len(positive_sites)
#end method

def num_positive_sites(json_data, pvalueThreshold, Positive=True):
    data = json_data["MLE"]["content"]["0"]
    columns = json_data["MLE"]["headers"]
    headers = [x[0] for x in columns]
    df = pd.DataFrame(data, columns=headers, dtype = float)
    df["omega"] = df["beta"] / df["alpha"]
    df_results = df[df["p-value"] <= pvalueThreshold]
    
    # 
    negative_sites = df_results[df_results["omega"] < 1.0]
    return len(negative_sites)


def num_selected_sites(json_data, pvalueThreshold, Positive=True):
    columns = json_data["MLE"]["headers"]
    headers = [x[0] for x in columns]
    
    data = json_data["MLE"]["content"]["0"]
    df = pd.DataFrame(data, columns=headers, dtype = float)
    df["omega"] = df["beta"] / df["alpha"]
    df_results = df[df["p-value"] <= pvalueThreshold]
    
    # boolean
    if Positive == True:
        positive_sites = df_results[df_results["omega"] > 1.0]
        return len(positive_sites)
    elif Positive == False:
        negative_sites = df_results[df_results["omega"] < 1.0]
        return len(negative_sites)
    else:
        return 0
    #end if
#end method


In [40]:
df_dict = {}
count = 0

for file in JSON_FILES:
    json_data = get_JSONData(file)
    
    basename = os.path.basename(file)
    df_dict[count] = {"Filename": basename}
    # Number of sites
    sites = json_data["input"]["number of sites"]
    # Number of sequences
    seqs = json_data["input"]["number of sequences"] 
    
    df_dict[count].update({"N": seqs,
                          "Num.Codons": sites})
    
    x = num_selected_sites(json_data, pvalueThreshold, True)
    
    y = num_selected_sites(json_data, pvalueThreshold, False)
    
    #x = 1
    #y = 1
    
    df_dict[count].update({"Num.Positive_Sites": x,
                          "Num.Negative_Sites": y})
    
    count += 1

#end for

In [47]:
df = pd.DataFrame.from_dict(df_dict, orient="index")
df.index += 1

df

,Filename,N,Num.Codons,Num.Positive_Sites,Num.Negative_Sites
1,RACCOON_G_FEL.json,241,552,4,349
2,RACCOON_M_FEL.json,131,203,0,116
3,RACCOON_L_FEL.json,142,2129,2,1341
4,HUMAN_G_FEL.json,121,528,1,415
5,DOG_L_FEL.json,384,2423,12,1773
6,DOG_M_FEL.json,232,227,1,166
7,DOG_G_FEL.json,1421,661,5,463
8,HUMAN_L_FEL.json,30,2129,1,1657
9,HUMAN_M_FEL.json,25,203,0,132
10,BAT_N_FEL.json,957,622,8,413


In [48]:
df = df.sort_values(by=['Filename'])
df = df.reset_index()
#df.drop('index', axis=1, inplace=True)
df.index += 1
df.drop('index', axis=1, inplace=True)
df

,Filename,N,Num.Codons,Num.Positive_Sites,Num.Negative_Sites
1,BAT_G_FEL.json,407,575,3,422
2,BAT_L_FEL.json,125,2129,2,1815
3,BAT_M_FEL.json,71,203,0,157
4,BAT_N_FEL.json,957,622,8,413
5,BAT_P_FEL.json,261,310,3,215
6,CAT_G_FEL.json,89,566,1,411
7,CAT_L_FEL.json,17,2129,2,1354
8,CAT_M_FEL.json,15,203,0,104
9,CAT_N_FEL.json,114,489,1,391
10,CAT_P_FEL.json,27,341,1,149


In [72]:
# Statistics

df["Percent.Positive_Sites"] = round((df["Num.Positive_Sites"] / df["Num.Codons"]) * 100, 6)
df["Percent.Negative_Sites"] = round((df["Num.Negative_Sites"] / df["Num.Codons"]) * 100, 6)

In [73]:
print(df.to_markdown())

|    | Filename           |    N |   Num.Codons |   Num.Positive_Sites |   Num.Negative_Sites |   Percent.Positive_Sites |   Percent.Negative_Sites |
|---:|:-------------------|-----:|-------------:|---------------------:|---------------------:|-------------------------:|-------------------------:|
|  1 | BAT_G_FEL.json     |  407 |          575 |                    3 |                  422 |                 0.521739 |                  73.3913 |
|  2 | BAT_L_FEL.json     |  125 |         2129 |                    2 |                 1815 |                 0.093941 |                  85.2513 |
|  3 | BAT_M_FEL.json     |   71 |          203 |                    0 |                  157 |                 0        |                  77.3399 |
|  4 | BAT_N_FEL.json     |  957 |          622 |                    8 |                  413 |                 1.28617  |                  66.3987 |
|  5 | BAT_P_FEL.json     |  261 |          310 |                    3 |                  215 |     